# Convert Endeavour Results in nCop weights

Input: 
 - Endeavour Results
 - nCop weights example

Output:
 - nCop weights

In [ ]:
import pandas as pd
import os

In [ ]:
path_nCopWeightExample = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/nCopWeightExample/weights.txt'
path_EndeavourResults = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/EndeavourResults/'
path_nCopWeights = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/nCopWeights/'

path_mafs_nCop = '/content/drive/MyDrive/Colab Notebooks/mafs_nCop/'
path_nets = '/content/drive/MyDrive/Colab Notebooks/redes/'

# Paths Endeavour Results Mutation Frequence, 3%, 5%, 10%
path_EndeavourResults3p = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/EndeavourResultsFrequence/3p/'
path_EndeavourResults5p = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/EndeavourResultsFrequence/5p/'
path_EndeavourResults10p = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/EndeavourResultsFrequence/10p/'

path_nCopWeights3p = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/nCopWeightsFrequence/3p/'
path_nCopWeights5p = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/nCopWeightsFrequence/5p/'
path_nCopWeights10p = '/content/drive/MyDrive/Colab Notebooks/enriquecimento/Endeavour/nCopWeightsFrequence/10p/'


In [ ]:
"""
Gera uma lista com o nome de todos os arquivos do diretório (mafs ou redes)
"""
def file_list_in_dir(path):
    l_files = []
    for _, _, file in os.walk(path):
        l_files = file
    return l_files

"""
Abre todos os mafs do diretório
"""    
def open_mafs(l_mafs_name):
    l_mafs_csv = []
    for maf in l_mafs_name:
        df = pd.read_csv(path+maf, sep='\t', usecols=["Hugo_Symbol", "Tumor_Sample_Barcode"])
        l_mafs_csv.append(df)
    return l_mafs_csv

In [ ]:
""" 
Cria um arquivo com a junção da saída do endeavour e arquivo de pesos (weight.txt) do nCop
"""
def altera_pvalue_gene(df_weights, df_enric):

    df_weights_novo = df_weights.copy()

    count = 0
    for gene_name in df_enric["Gene name"]:

        """contador inicio"""
        """
        count+=1
        if count > 5:
            break
        """
        """contador inicio"""

        p_value_enric = df_enric.loc[df_enric["Gene name"] == gene_name]["P-value"].values
        p_value_weights_antes = df_weights_novo.loc[df_weights_novo["Gene name"] == gene_name]["P-value"].values
        # Altera o p-value do weights: aqui coloquei o indice 0 pois pode haver dois genes com p-values diferentes no enriquecimento, assim pega só o primeiro
        df_weights_novo.loc[df_weights_novo["Gene name"] == gene_name, "P-value"] = float(p_value_enric[0])
        p_value_weights_depois = df_weights_novo.loc[df_weights_novo["Gene name"] == gene_name]["P-value"].values

        """ verifica o tipo de dados de cada coluna de cada dataframe """
        #print(df_weights_novo.info())
        #print(df_enric.info())
        
        """
        print("Gene: " + gene_name)
        print("P-value enric " + str(p_value_enric))
        print("P-value weights antes " + str(p_value_weights_antes))
        print("P-value weights depois " + str(p_value_weights_depois))
        print("**************************************************")
        """

    return df_weights_novo

"""
Gerar uma lista ponderada com dados de enriquecimento, em cima de todos os 19.454 genes da lista de weigth do nCop
"""
def enriquece_weights_ncop(path_nCopWeightExample, path_EndeavourResults, path_nCopWeights):
    
    endeavourResults = file_list_in_dir(path_EndeavourResults)
    
    for endeavourResultNameFile in endeavourResults:
        dfnCopWeightExample = pd.read_csv(path_nCopWeightExample, sep="\s+", header=None, names=["Gene name", "P-value"]) 
        dfEndeavourResults = pd.read_csv(path_EndeavourResults+endeavourResultNameFile, sep=',')
        dfnCopWeightExampleCopy = dfnCopWeightExample.copy()
        """
        Converte as colunas p-value para numérico
        """
        dfnCopWeightExampleCopy["P-value"] = pd.to_numeric(dfnCopWeightExampleCopy["P-value"])
        dfEndeavourResults["P-value"] = pd.to_numeric(dfEndeavourResults["P-value"])

        """
        Altera o p-value do df_weights nCop de acordo com o p-value do enriquecimento
        """
        dfnCopWeightExampleNew = altera_pvalue_gene(dfnCopWeightExampleCopy, dfEndeavourResults)
            
        #print_info("Verifica se tem nulos", dfnCopWeightExampleNew.isnull().sum())

        # Salva o novo arquivo de pesos gerado
        dfnCopWeightExampleNewName = endeavourResultNameFile.replace('endeavour_result', 'weights')
        dfnCopWeightExampleNewName = dfnCopWeightExampleNewName.replace('.csv', '.txt')
        dfnCopWeightExampleNewName = path_nCopWeights+dfnCopWeightExampleNewName
        dfnCopWeightExampleNew.to_csv(dfnCopWeightExampleNewName, sep="\t", header=None, index=False)
        





In [ ]:
endeavourResults = file_list_in_dir(path_EndeavourResults)
print(endeavourResults)
enriquece_weights_ncop(path_nCopWeightExample, path_EndeavourResults, path_nCopWeights)



['endeavour_result_multinet_gbm_reactome.csv', 'endeavour_result_multinet_brca_reactome.csv', 'endeavour_result_irefindex_gbm_reactome.csv', 'endeavour_result_irefindex_brca_reactome.csv', 'endeavour_result_hprd_gbm_reactome.csv', 'endeavour_result_hprd_brca_reactome.csv', 'endeavour_result_hint_gbm_reactome.csv', 'endeavour_result_hint_brca_reactome.csv']


In [ ]:
# Frequence mutations: 3p
endeavourResults3p = file_list_in_dir(path_EndeavourResults3p)
print(endeavourResults3p)
enriquece_weights_ncop(path_nCopWeightExample, path_EndeavourResults3p, path_nCopWeights3p)
# Frequence mutations: 5p
endeavourResults5p = file_list_in_dir(path_EndeavourResults5p)
print(endeavourResults5p)
enriquece_weights_ncop(path_nCopWeightExample, path_EndeavourResults5p, path_nCopWeights5p)
# Frequence mutations: 10p
endeavourResults10p = file_list_in_dir(path_EndeavourResults10p)
print(endeavourResults10p)
enriquece_weights_ncop(path_nCopWeightExample, path_EndeavourResults10p, path_nCopWeights10p)

['endeavour_result_ACC.csv', 'endeavour_result_BLCA.csv', 'endeavour_result_BRCA.csv', 'endeavour_result_CESC.csv', 'endeavour_result_COAD.csv', 'endeavour_result_GBM.csv', 'endeavour_result_HNSC.csv', 'endeavour_result_KICH.csv', 'endeavour_result_KIRC.csv', 'endeavour_result_KIRP.csv', 'endeavour_result_LGG.csv', 'endeavour_result_LIHC.csv', 'endeavour_result_LUAD.csv', 'endeavour_result_LUSC.csv', 'endeavour_result_OV.csv', 'endeavour_result_PAAD.csv', 'endeavour_result_PCPG.csv', 'endeavour_result_PRAD.csv', 'endeavour_result_SKCM.csv', 'endeavour_result_STAD.csv', 'endeavour_result_THCA.csv', 'endeavour_result_UCEC.csv', 'endeavour_result_UCS.csv']
['endeavour_result_ACC.csv', 'endeavour_result_BLCA.csv', 'endeavour_result_BRCA.csv', 'endeavour_result_CESC.csv', 'endeavour_result_COAD.csv', 'endeavour_result_GBM.csv', 'endeavour_result_HNSC.csv', 'endeavour_result_KICH.csv', 'endeavour_result_KIRC.csv', 'endeavour_result_KIRP.csv', 'endeavour_result_LGG.csv', 'endeavour_result_LIH